# Google Drive 연결

In [13]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 라이브러리

In [14]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (
    roc_auc_score, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, classification_report
)

# 데이터 가져오기

In [15]:
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/Multicampus-8/4_머신러닝_딥러닝/프로젝트/'

train_df = pd.read_csv(DATA_PATH + "dataset/train.csv")
test_df = pd.read_csv(DATA_PATH + "dataset/test.csv")
target = "diagnosed_diabetes"

# 데이터 분리

In [16]:
X = train_df.drop(["id", target], axis=1, errors="ignore")
y = train_df[target]

X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 전처리 설계

In [17]:
# Ordinal Encoding
target_ordinal = ['income_level'] # 순서형 자료
income_cats = ['Low', 'Lower-Middle', 'Middle', 'Upper-Middle', 'High']

# OneHot Encoding 컬럼들
categorical_features = X_train_raw.select_dtypes(include=['object']).columns.tolist()

# 순서형 자료 컬럼은 제거
# 집합 연산
categorical_features = list(set(categorical_features) - set(target_ordinal))

# 수치형 자료에서 bmi 제거하기로 함
# 회귀분석 진행했더니, waist_to_hip_ratio > bmi, 영향도가 높음
numeric_features = X_train_raw.select_dtypes(include=np.number).columns.tolist()

remove_features = ['bmi']
numeric_features = list(set(numeric_features) - set(remove_features))

# 전처리기 생성
preprocessor = ColumnTransformer(
    transformers=[("num", StandardScaler(), numeric_features),
                  ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features), ("ord", OrdinalEncoder(categories=[income_cats], handle_unknown="use_encoded_value", unknown_value=-1), target_ordinal)]
)

preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['age', 'hypertension_history',
                                  'ldl_cholesterol',
                                  'screen_time_hours_per_day',
                                  'waist_to_hip_ratio', 'systolic_bp',
                                  'alcohol_consumption_per_week',
                                  'family_history_diabetes',
                                  'sleep_hours_per_day', 'diastolic_bp',
                                  'cholesterol_total',
                                  'physical_activity_minutes_per_week',
                                  'triglycerides', 'diet_score', 'he...
                                  'cardiovascular_history',
                                  'hdl_cholesterol']),
                                ('cat',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['education_level', 'ethnicity', 'gender',
                                  'employment_status', 'smoking_status']),
                                ('ord',
                                 OrdinalEncoder(categories=[['Low',
                                                             'Lower-Middle',
                                                             'Middle',
                                                             'Upper-Middle',
                                                             'High']],
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['income_level'])])

# 모델 개발 및 하이퍼파라미터 설정
- 랜덤서치는 확률분포나 범위 지정 가능

1. 클래스 불균형 처리를 위한 비율 계산 (`ratio`)
- **의미**: 학습 데이터(`y_train`) 내 다수 클래스(0: 정상) 대비 소수 클래스(1: 당뇨)의 비율을 산출합니다.
- **용도**: 데이터 불균형이 심할 때, 모델이 소수 클래스에 더 가중치를 두어 학습하도록 돕는 지표로 활용됩니다.



2. 로지스틱 회귀(Logistic Regression) 설정
- **의미**: 선형 분류 모델을 기반으로 하이퍼파라미터 탐색 범위를 정의합니다.
- **`class_weight="balanced"`**: 데이터 빈도에 반비례하여 가중치를 자동 부여하여 불균형을 해결합니다.
- **`model__C`**: 규제(Regularization) 강도의 역수입니다. 값이 작을수록 더 강력한 규제를 적용합니다.

3. 결정 트리(Decision Tree) 설정
- **의미**: 데이터 규칙을 나무 구조로 시각화하는 비선형 모델의 탐색 공간을 정의합니다.
- **`model__max_depth`**: 나무의 최대 깊이를 제한하여 모델이 너무 복잡해지는 것(과적합)을 방지합니다.
- **`model__min_samples_leaf`**: 리프 노드(말단)가 되기 위한 최소 샘플 수로, 가지치기의 역할을 수행합니다.



4. XGBoost 설정
- **의미**: 성능이 우수한 그래디언트 부스팅(GBDT) 알고리즘의 최적 파라미터를 탐색합니다.
- **`scale_pos_weight=ratio`**: 앞서 계산한 클래스 비율을 적용하여 양성 클래스(1)에 대한 가중치를 직접 조절합니다.
- **`model__learning_rate`**: 각 단계에서 오차를 보정하는 보폭을 결정하며, `max_depth` 및 `n_estimators`와 결합하여 성능을 최적화합니다.
- **`model__subsample`**: 나무를 만들 때 사용하는 무작위 샘플링 비율로, 모델의 일반화 성능을 높입니다.

In [18]:
ratio = (y_train == 0).sum() / (y_train == 1).sum()
print(ratio)

model_configs = [
    {
        "name": "Logistic Regression",
        "model": LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
        "params": {
            "model__C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], # 규제 강도 무작위 탐색
            "model__penalty": ['l2']
        }
    },
    {
        "name": "Decision Tree",
        "model": DecisionTreeClassifier(class_weight="balanced", random_state=42),
        "params": {
            "model__max_depth": np.arange(3, 16), # 3~15 사이 무작위 탐색
            "model__min_samples_leaf": [10, 20, 30, 50, 100]
        }
    },
    {
        "name": "XGBoost",
        "model": XGBClassifier(eval_metric="logloss", random_state=42, n_jobs=-1, scale_pos_weight=ratio),
        "params": {
            "model__n_estimators": [100, 200, 300, 400],
            "model__learning_rate": [0.01, 0.05, 0.1, 0.2],
            "model__max_depth": [3, 4, 5, 6],
            "model__subsample": [0.7, 0.8, 0.9]
        }
    },
    {
        "name": "LightGBM",
        "model": LGBMClassifier(random_state=42, n_jobs=-1, scale_pos_weight=ratio, verbosity=-1),
        "params": {
            "model__n_estimators": [100, 200, 300, 400],
            "model__learning_rate": [0.01, 0.05, 0.1, 0.2],
            "model__subsample": [0.7, 0.8, 0.9],
            "model__max_depth": [3, 4, 5, 6]
        }
    }
]

0.6043730625762793


# Random Search 학습 진행

In [19]:
import time
from datetime import timedelta
import warnings
from tqdm.auto import tqdm # 진행 바 라이브러리

warnings.filterwarnings('ignore', category=UserWarning)

# 1. 교차 검증 및 결과 저장 준비
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []

# 2. 전체 모델 루프에 tqdm 적용 (전체 진행률 확인)
for config in tqdm(model_configs, desc="전체 모델 최적화 진행 중"):
    print(f"\n [{config['name']}] 랜덤 서치 탐색을 시작합니다...")

    # 시간 측정 시작
    start_time = time.perf_counter()

    pipeline = Pipeline([("prep", preprocessor), ("model", config['model'])])

    # RandomizedSearchCV 설정
    # verbose=1: 간단한 진행 상황 출력 (각 파라미터 조합 시도 시 출력)
    # n_jobs=-1: 병렬 처리 활성화
    search = RandomizedSearchCV(
        pipeline,
        param_distributions=config['params'],
        n_iter=10,
        cv=cv,
        scoring='roc_auc',
        n_jobs=-1,
        random_state=42,
        verbose=1 # <--- 실행 중인 상태를 로그로 보여줍니다.
    )

    # 모델 학습 (실제 탐색 수행)
    search.fit(X_train_raw, y_train)

    # 시간 측정 종료 및 변환
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    elapsed_hms = str(timedelta(seconds=int(elapsed_time)))

    # 검증셋 평가
    val_probs = search.predict_proba(X_val_raw)[:, 1]
    val_auc = roc_auc_score(y_val, val_probs)
    val_pred = search.predict(X_val_raw)

    # 상세 지표 계산
    tn, fp, fn, tp = confusion_matrix(y_val, val_pred).ravel()

    print(f"[{config['name']}] 탐색 완료!")
    print(f"소요 시간: {elapsed_hms} | Best CV AUC: {search.best_score_:.4f} | Val AUC: {val_auc:.4f}")

    # 결과 저장
    results.append({
        "Model": config['name'],
        "AUC": roc_auc_score(y_val, val_probs),
        "Accuracy": accuracy_score(y_val, val_pred),
        "Precision": precision_score(y_val, val_pred),
        "Recall": recall_score(y_val, val_pred),
        "F1": f1_score(y_val, val_pred),
        "TP": tp, "TN": tn, "FP": fp, "FN": fn,
        "Best_CV_AUC": search.best_score_,
        "Val_AUC": val_auc,
        "Time": elapsed_hms,
        "Best_Params": search.best_params_
    })

전체 모델 최적화 진행 중:   0%|          | 0/4 [00:00<?, ?it/s]


 [Logistic Regression] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[Logistic Regression] 탐색 완료!
소요 시간: 0:01:22 | Best CV AUC: 0.6937 | Val AUC: 0.6940

 [Decision Tree] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Decision Tree] 탐색 완료!
소요 시간: 0:05:45 | Best CV AUC: 0.6908 | Val AUC: 0.6917

 [XGBoost] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[XGBoost] 탐색 완료!
소요 시간: 0:14:53 | Best CV AUC: 0.7239 | Val AUC: 0.7241

 [LightGBM] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] 탐색 완료!
소요 시간: 0:25:48 | Best CV AUC: 0.7251 | Val AUC: 0.7252


# 결과 확인

In [20]:
# 결과 확인
import pandas as pd
results_df = pd.DataFrame(results)
results_df

,Model,AUC,Accuracy,Precision,Recall,F1,TP,TN,FP,FN,Best_CV_AUC,Val_AUC,Time,Best_Params
0,Logistic Regression,0.694032,0.626671,0.754047,0.595169,0.665253,51935,35799,16940,35326,0.693716,0.694032,0:01:22,"{'model__penalty': 'l2', 'model__C': 100.0}"
1,Decision Tree,0.691713,0.620529,0.759073,0.573074,0.653089,50007,36867,15872,37254,0.690812,0.691713,0:05:45,"{'model__min_samples_leaf': 10, 'model__max_de..."
2,XGBoost,0.724062,0.654186,0.771981,0.631794,0.694888,55131,36455,16284,32130,0.723871,0.724062,0:14:53,"{'model__subsample': 0.9, 'model__n_estimators..."
3,LightGBM,0.725170,0.653671,0.773429,0.628459,0.693449,54840,36674,16065,32421,0.725150,0.725170,0:25:48,"{'model__subsample': 0.9, 'model__n_estimators..."


In [21]:
display(results_df[["Model", "Best_CV_AUC", "Val_AUC", "Time"]])

,Model,Best_CV_AUC,Val_AUC,Time
0,Logistic Regression,0.693716,0.694032,0:01:22
1,Decision Tree,0.690812,0.691713,0:05:45
2,XGBoost,0.723871,0.724062,0:14:53
3,LightGBM,0.725150,0.725170,0:25:48


In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 성능(AUC)과 시간(초 단위) 비교 차트
results_df['Seconds'] = results_df['Time'].apply(lambda x: sum(int(a) * 60**i for i, a in enumerate(reversed(x.split(':')))))

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=results_df['Model'], y=results_df['Val_AUC'], name="Validation AUC"), secondary_y=False)
fig.add_trace(go.Scatter(x=results_df['Model'], y=results_df['Seconds'], name="Time (seconds)", mode='lines+markers'), secondary_y=True)

fig.update_layout(title_text="모델별 성능 및 최적화 소요 시간 비교")
fig.update_yaxes(title_text="AUC Score", secondary_y=False)
fig.update_yaxes(title_text="Time (s)", secondary_y=True)
fig.show()

# 제출 파일 생성코드

In [23]:
# 1. 랜덤 서치 결과 중 가장 성능(Val_AUC)이 좋은 모델 정보 가져오기
best_model_info = max(results, key=lambda x: x['Val_AUC'])
best_model_name = best_model_info['Model']
best_params = best_model_info['Best_Params']

print(f"🏆 최종 선택된 모델: {best_model_name}")
print(f"⚙️ 최적 하이퍼파라미터: {best_params}")

# 2. 최적 모델 객체 찾기 및 파라미터 적용
# model_configs 리스트에서 이름이 같은 모델 설정을 찾습니다.
selected_config = next(config for config in model_configs if config["name"] == best_model_name)
final_model_obj = selected_config["model"]

# 파생된 파이프라인 생성 (전처리기 + 선택된 모델)
final_pipeline = Pipeline([
    ("prep", preprocessor),
    ("model", final_model_obj)
])

# RandomizedSearchCV에서 찾은 'best_params'를 파이프라인에 직접 주입합니다.
final_pipeline.set_params(**best_params)

# 3. 전체 데이터로 재학습 (Train + Validation 합쳐서 학습하면 성능이 더 좋아집니다)
# 강의용이라면 간단하게 X_train_raw만 사용해도 무방합니다.
print("최적 파라미터로 모델 재학습 중...")
final_pipeline.fit(X_train_raw, y_train)

# 4. 테스트 데이터 예측
test_X = test_df.drop(["id"], axis=1, errors="ignore")
test_proba = final_pipeline.predict_proba(test_X)[:, 1]

# 5. 제출 데이터프레임 구성
submission = pd.DataFrame({
    "id": test_df["id"] if "id" in test_df.columns else np.arange(len(test_df)),
    "diagnosed_diabetes": test_proba
})

# 6. 파일 저장 및 확인
file_name = f"submission_best_{best_model_name.lower().replace(' ', '_')}.csv"
save_path = DATA_PATH + "output/" + file_name
submission.to_csv(save_path, index=False)

print(f"제출 파일 저장 완료: {save_path}")
display(submission.head())

🏆 최종 선택된 모델: LightGBM
⚙️ 최적 하이퍼파라미터: {'model__subsample': 0.9, 'model__n_estimators': 400, 'model__max_depth': 6, 'model__learning_rate': 0.1}
최적 파라미터로 모델 재학습 중...
제출 파일 저장 완료: /content/drive/MyDrive/Colab Notebooks/Multicampus-8/4_머신러닝_딥러닝/프로젝트/output/submission_best_lightgbm.csv


,id,diagnosed_diabetes
0,700000,0.377305
1,700001,0.509006
2,700002,0.703246
3,700003,0.297680
4,700004,0.901096


In [24]:
# 5. 파일 다운로드 (Colab)
from google.colab import files
files.download(save_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>